In [83]:
# Import libraries
import pandas as pd
import pydeck as pdk

In [84]:
# Set view options
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [93]:
# Read in hop teaming dataset
hop_teaming_provider_to_facility = pd.read_csv('../data/hop_teaming_provider_to_facility.csv')

In [94]:
hop_teaming_provider_to_facility

,from_npi,referral_id,entity_from,name_from,organization_from,taxonomy_code_from,classification_from,specialization_from,address_line1_from,address_line2_from,city_from,state_from,zip_from,to_npi,entity_to,name_to,organization_to,taxonomy_code_to,classification_to,specialization_to,address_line1_to,address_line2_to,city_to,state_to,zip_to,patient_count,transaction_count,average_day_wait,std_day_wait,address_from,address_to,lat_from,lng_from,lat_to,lng_to,distance_miles,from_communityid,to_communityid,affiliation_from,affiliation_to
0,1821080961,18854880,1.0,"SHERWOOD, WILLIAM",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,91,238,9.924,40.266,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624,20624,NOT IN NASHVILLE,NO MAJOR AFFILIATION/OTHER
1,1841282779,131966153,1.0,"RHODY, KEVIN",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,94,236,7.695,36.447,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624,20624,NOT IN NASHVILLE,NO MAJOR AFFILIATION/OTHER
2,1770575607,146169993,1.0,"COOPER, STEVEN",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1013012616,2.0,NaN,"CRIPPS, HOOPER & RHODY, PLLC",207Q00000X,Family Medicine,NaN,400 E PUBLIC SQUARE,NaN,ALEXANDRIA,TN,37012,115,320,2.725,18.756,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","400 E PUBLIC SQUARE, ALEXANDRIA, TN 37012",35.969171,-85.807472,36.077559,-86.033322,14.665166,20624,20624,NOT IN NASHVILLE,NO MAJOR AFFILIATION/OTHER
3,1821080961,19037011,1.0,"SHERWOOD, WILLIAM",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1093741464,2.0,NaN,"ADVANCED DIAGNOSTIC IMAGING, PC",207R00000X,Internal Medicine,NaN,3024 BUSINESS PARK CIR,NaN,GOODLETTSVILLE,TN,37072,155,199,42.241,64.876,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072",35.969171,-85.807472,36.332108,-86.700830,55.754781,20624,9794,NOT IN NASHVILLE,NO MAJOR AFFILIATION/OTHER
4,1841282779,132118112,1.0,"RHODY, KEVIN",NaN,207Q00000X,Family Medicine,NaN,302 N CONGRESS BLVD,NaN,SMITHVILLE,TN,37166,1093741464,2.0,NaN,"ADVANCED DIAGNOSTIC IMAGING, PC",207R00000X,Internal Medicine,NaN,3024 BUSINESS PARK CIR,NaN,GOODLETTSVILLE,TN,37072,162,218,47.743,69.581,"302 N CONGRESS BLVD, SMITHVILLE, TN 37166","3024 BUSINESS PARK CIR, GOODLETTSVILLE, TN 37072",35.969171,-85.807472,36.332108,-86.700830,55.754781,20624,9794,NOT IN NASHVILLE,NO MAJOR AFFILIATION/OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41223,1154635118,162633678,1.0,"YORK, DEBRA",NaN,363LP0808X,Nurse Practitioner,Psychiatric/Mental Health,922 HARPETH VALLEY PL STE 2,NaN,NASHVILLE,TN,37221,1457749681,2.0,NaN,"THE ART OF HEALING, P.C.",363LP0808X,Nurse Practitioner,Psychiatric/Mental Health,922 HARPETH VALLEY PL STE 2,NaN,NASHVILLE,TN,37221,20,73,0.000,0.000,"922 HARPETH VALLEY PL STE 2, NASHVILLE, TN 37221","922 HARPETH VALLEY PL STE 2, NASHVILLE, TN 37221",36.083142,-86.956284,36.083142,-86.956284,0.000000,23842,23842,NO MAJOR AFFILIATION/OTHER,NO MAJOR AFFILIATION/OTHER
41224,1104065630,151563399,1.0,"MESECK, JACOB",NaN,111N00000X,Chiropractor,NaN,2199 FAIRVIEW BLVD,NaN,FAIRVIEW,TN,37062,1710288980,2.0,NaN,MESECK FAMILY CHIROPRACTIC LLC,261QH0100X,Clinic/Center,Health Service,865 BELLEVUE RD,APT. U2,NASHVILLE,TN,37221,28,267,0.000,0.000,"2199 FAIRVIEW BLVD, FAIRVIEW, TN 37062","865 BELLEVUE RD, APT. U2, NASHVILLE, TN 37221",35.973919,-87.127618,36.073955

In [95]:
# Create affiliation dataframe for rgb values
rgb = {'affiliation': ['VANDERBILT', 'HCA', 'SAINT THOMAS ASCENSION', 'LIFEPOINT', 'CHS'], 
       'r': [154, 226, 28, 95, 0],
       'g': [120, 89, 66, 96, 108],
       'b': [61, 41, 151, 98, 103]}

rgb_df = pd.DataFrame(data = rgb)

In [96]:
rgb_df

,affiliation,r,g,b
0,VANDERBILT,154,120,61
1,HCA,226,89,41
2,SAINT THOMAS ASCENSION,28,66,151
3,LIFEPOINT,95,96,98
4,CHS,0,108,103


In [100]:
# Merge hop teaming data with rgb dataframe, aggregate patient counts by facility geocoordinates and affiliation
referrals_to_hospitals = hop_teaming_provider_to_facility.merge(rgb_df.add_suffix('_to'), on = 'affiliation_to') \
    .groupby(['lat_to', 'lng_to', 'affiliation_to', 'r_to', 'g_to', 'b_to'], as_index = False)['patient_count'].sum()

# Rename 'SAINT THOMAS ASCENSION' to 'ASCENSION SAINT THOMAS' 
referrals_to_hospitals['affiliation_to'] = referrals_to_hospitals['affiliation_to'].str.replace('SAINT THOMAS ASCENSION', 'ASCENSION SAINT THOMAS')

# Add commas to patient counts and store as string
referrals_to_hospitals['patient_count_str'] = referrals_to_hospitals.apply(lambda col: '{:,}'.format(col['patient_count']), axis = 1)

# Create column layer
column_layer = pdk.Layer(
    type = 'ColumnLayer',
    data = referrals_to_hospitals,
    get_position = ['lng_to', 'lat_to'],
    get_elevation = 'patient_count',
    elevation_scale = 40000 / referrals_to_hospitals['patient_count'].max(),
    radius = 500,
    get_fill_color = ['r_to', 'g_to', 'b_to', 150],
    pickable = True
)

# Set intial view
view = pdk.data_utils.compute_view(referrals_to_hospitals[['lng_to', 'lat_to']])
view.pitch = 60
view.bearing = 0
view.zoom = 8
view.min_zoom = 7

# Create tooltip
tooltip = {
    'html': '<b>{patient_count_str}</b> patient referrals to <b>{affiliation_to}</b>',
    'style': {'background': 'grey', 'color': 'white', 'font-family': 'Lato, Arial', 'z-index': '10000'},
}

# Render map
r = pdk.Deck(
    layers = column_layer,
    initial_view_state = view,
    tooltip = tooltip,
    map_style = pdk.map_styles.CARTO_ROAD,
)

# Save map to html
r.to_html('../output/referrals_to_hospitals.html')

In [87]:
# Filter internal medicine providers, merge with rgb dataframe, aggregate patient counts by provider geocoordinates and facility affiliation
referrals_from_internal_medicine = hop_teaming_provider_to_facility.query('classification_from == "Internal Medicine"') \
    .merge(rgb_df.add_suffix('_to'), on = 'affiliation_to') \
    .groupby(['lat_from', 'lng_from', 'affiliation_to', 'r_to', 'g_to', 'b_to'], as_index = False)['patient_count'].sum()

# Rename 'SAINT THOMAS ASCENSION' to 'ASCENSION SAINT THOMAS' 
referrals_from_internal_medicine['affiliation_to'] = referrals_from_internal_medicine['affiliation_to'].str.replace('SAINT THOMAS ASCENSION', 'ASCENSION SAINT THOMAS')

# Add commas to patient counts and store as string
referrals_from_internal_medicine['patient_count_str'] = referrals_from_internal_medicine.apply(lambda col: '{:,}'.format(col['patient_count']), axis = 1)

# Create column layer
column_layer = pdk.Layer(
    type = 'ColumnLayer',
    data = referrals_from_internal_medicine,
    get_position = ['lng_from', 'lat_from'],
    get_elevation = 'patient_count',
    elevation_scale = 40000 / referrals_from_internal_medicine['patient_count'].max(),
    radius = 500,
    get_fill_color = ['r_to', 'g_to', 'b_to', 150],
    pickable = True
)

# Set initial view
view = pdk.data_utils.compute_view(referrals_from_internal_medicine[['lng_from', 'lat_from']])
view.pitch = 60
view.bearing = 0
view.zoom = 8
view.min_zoom = 7

# Create tooltip
tooltip = {
    'html': '<b>{patient_count_str}</b> patient referrals to <b>{affiliation_to}</b>',
    'style': {'background': 'grey', 'color': 'white', 'font-family': 'Lato, Arial', 'z-index': '10000'},
}

# Render map
r = pdk.Deck(
    layers = column_layer,
    initial_view_state = view,
    tooltip = tooltip,
    map_style = pdk.map_styles.CARTO_ROAD,
)

# Save map to html
r.to_html('../output/referrals_from_internal_medicine.html')

In [88]:
# Filter non-internal medicine providers, merge with rgb dataframe, aggregate patient counts by provider geocoordinates and facility affiliation
referrals_from_noninternal_medicine = hop_teaming_provider_to_facility.query('classification_from != "Internal Medicine"') \
    .merge(rgb_df.add_suffix('_to'), on = 'affiliation_to') \
    .groupby(['lat_from', 'lng_from', 'affiliation_to', 'r_to', 'g_to', 'b_to'], as_index = False)['patient_count'].sum()

# Rename 'SAINT THOMAS ASCENSION' to 'ASCENSION SAINT THOMAS' 
referrals_from_noninternal_medicine['affiliation_to'] = referrals_from_noninternal_medicine['affiliation_to'].str.replace('SAINT THOMAS ASCENSION', 'ASCENSION SAINT THOMAS')

# Add commas to patient counts and store as string
referrals_from_noninternal_medicine['patient_count_str'] = referrals_from_noninternal_medicine.apply(lambda col: '{:,}'.format(col['patient_count']), axis = 1)

# Create column layer
column_layer = pdk.Layer(
    type = 'ColumnLayer',
    data = referrals_from_noninternal_medicine,
    get_position = ['lng_from', 'lat_from'],
    get_elevation = 'patient_count',
    elevation_scale = 40000 / referrals_from_noninternal_medicine['patient_count'].max(),
    radius = 500,
    get_fill_color = ['r_to', 'g_to', 'b_to', 150],
    pickable = True
)

# Set intial view
view = pdk.data_utils.compute_view(referrals_from_noninternal_medicine[['lng_from', 'lat_from']])
view.pitch = 60
view.bearing = 0
view.zoom = 8
view.min_zoom = 7

# Create tooltip
tooltip = {
    'html': '<b>{patient_count_str}</b> patient referrals to <b>{affiliation_to}</b>',
    'style': {'background': 'grey', 'color': 'white', 'font-family': 'Lato, Arial', 'z-index': '10000'},
}

# Render map
r = pdk.Deck(
    layers = column_layer,
    initial_view_state = view,
    tooltip = tooltip,
    map_style = pdk.map_styles.CARTO_ROAD,
)

# Save map to html
r.to_html('../output/referrals_from_noninternal_medicine.html')

In [89]:
# Filter family medicine providers, merge with rgb dataframe, aggregate patient counts by provider geocoordinates and facility affiliation
referrals_from_family_medicine = hop_teaming_provider_to_facility.query('classification_from == "Family Medicine"') \
    .merge(rgb_df.add_suffix('_to'), on = 'affiliation_to') \
    .groupby(['lat_from', 'lng_from', 'affiliation_to', 'r_to', 'g_to', 'b_to'], as_index = False)['patient_count'].sum()

# Rename 'SAINT THOMAS ASCENSION' to 'ASCENSION SAINT THOMAS' 
referrals_from_family_medicine['affiliation_to'] = referrals_from_family_medicine['affiliation_to'].str.replace('SAINT THOMAS ASCENSION', 'ASCENSION SAINT THOMAS')

# Add commas to patient counts and store as string
referrals_from_family_medicine['patient_count_str'] = referrals_from_family_medicine.apply(lambda col: '{:,}'.format(col['patient_count']), axis = 1)

# Create column layer
column_layer = pdk.Layer(
    type = 'ColumnLayer',
    data = referrals_from_family_medicine,
    get_position = ['lng_from', 'lat_from'],
    get_elevation = 'patient_count',
    elevation_scale = 40000 / referrals_from_family_medicine['patient_count'].max(),
    radius = 500,
    get_fill_color = ['r_to', 'g_to', 'b_to', 150],
    pickable = True
)

# Set intial view
view = pdk.data_utils.compute_view(referrals_from_family_medicine[['lng_from', 'lat_from']])
view.pitch = 60
view.bearing = 0
view.zoom = 8
view.min_zoom = 7

# Create tooltip
tooltip = {
    'html': '<b>{patient_count_str}</b> patient referrals to <b>{affiliation_to}</b>',
    'style': {'background': 'grey', 'color': 'white', 'font-family': 'Lato, Arial', 'z-index': '10000'},
}

# Render map
r = pdk.Deck(
    layers = column_layer,
    initial_view_state = view,
    tooltip = tooltip,
    map_style = pdk.map_styles.CARTO_ROAD,
)

# Save map to html
r.to_html('../output/referrals_from_family_medicine.html')

In [90]:
# Filter orthopaedic surgery providers, merge with rgb dataframe, aggregate patient counts by provider geocoordinates and facility affiliation
referrals_from_orthopaedic_surgery = hop_teaming_provider_to_facility.query('classification_from == "Orthopaedic Surgery"') \
    .merge(rgb_df.add_suffix('_to'), on = 'affiliation_to') \
    .groupby(['lat_from', 'lng_from', 'affiliation_to', 'r_to', 'g_to', 'b_to'], as_index = False)['patient_count'].sum()

# Rename 'SAINT THOMAS ASCENSION' to 'ASCENSION SAINT THOMAS' 
referrals_from_orthopaedic_surgery['affiliation_to'] = referrals_from_orthopaedic_surgery['affiliation_to'].str.replace('SAINT THOMAS ASCENSION', 'ASCENSION SAINT THOMAS')

# Add commas to patient counts and store as string
referrals_from_orthopaedic_surgery['patient_count_str'] = referrals_from_orthopaedic_surgery.apply(lambda col: '{:,}'.format(col['patient_count']), axis = 1)

# Create column layer
column_layer = pdk.Layer(
    type = 'ColumnLayer',
    data = referrals_from_orthopaedic_surgery,
    get_position = ['lng_from', 'lat_from'],
    get_elevation = 'patient_count',
    elevation_scale = 40000 / referrals_from_orthopaedic_surgery['patient_count'].max(),
    radius = 500,
    get_fill_color = ['r_to', 'g_to', 'b_to', 150],
    pickable = True
)

# Set intial view
view = pdk.data_utils.compute_view(referrals_from_orthopaedic_surgery[['lng_from', 'lat_from']])
view.pitch = 60
view.bearing = 0
view.zoom = 8
view.min_zoom = 7

# Create tooltip
tooltip = {
    'html': '<b>{patient_count_str}</b> patient referrals to <b>{affiliation_to}</b>',
    'style': {'background': 'grey', 'color': 'white', 'font-family': 'Lato, Arial', 'z-index': '10000'},
}

# Render map
r = pdk.Deck(
    layers = column_layer,
    initial_view_state = view,
    tooltip = tooltip,
    map_style = pdk.map_styles.CARTO_ROAD,
)

# Save map to html
r.to_html('../output/referrals_from_orthopaedic_surgery.html')